In [3]:
import requests
from bs4 import BeautifulSoup
from langchain_openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
import os

In [13]:
os.environ["OPENAI_API_KEY"]= ""
LANGCHAIN_TRACING_V2="true"
LANGCHAIN_API_KEY= ""

In [5]:
def extract_page(url):
    text=requests.get(url)
    soup=BeautifulSoup(text.text,"html.parser")
    return soup.text

In [6]:
def language_detector(query):
    lang_url="https://ddragon.leagueoflegends.com/cdn/languages.json"
    text_langs=extract_page(lang_url).split('"')

    list_langs=[]
    for lang in range(1,len(text_langs),2):
        list_langs.append(text_langs[lang])

    determinate_lang=f"""You are an expert to determinate what language a text is in.
    You are an expert because you always check the whole text to determinate what overall language is used.
    You always provide a single language among the following list: {list_langs}
    You always provide only the language, nothing more.
    Here is a text:
    {query}"""

    lang=llm.invoke(determinate_lang).content
    return lang

In [7]:
def champion_detector(query):
    champ_url="https://ddragon.leagueoflegends.com/cdn/15.23.1/data/en_US/champion.json"
    champ_page=requests.get(champ_url).json()
    champs=[champ for champ in champ_page["data"]]

    determinate_champ=f"""You are an expert to determinate what League Of Legends champions are related to a text.
    You are an expert because you always check the list of League Of Legends champions and then you compare it to the text.
    You provide as many champions as mentionned in the text and present in this list: {champs}
    You always provide the list of related champions as followed: [\"champion_name1\",\"champion_name2\"] with as many names as related.
    Here is a text:
    {query}"""

    champions=llm.invoke(determinate_champ).content.split('"')
    champions_list=[]

    for champ in range(1,len(champions),2):
        champions_list.append(champions[champ])

    return champions_list

In [8]:
ATTRIBUTES_CHAMP=[
    "allytips",
    "enemytips",
    "tags",
    "partype",
    "info",
    "stats",
    "spells",
    "passive",
    "recommended"
]

def champ_infos(lang,champion):
    champ_url=f"https://ddragon.leagueoflegends.com/cdn/15.23.1/data/{lang}/champion/{champion}.json"
    champ_page=requests.get(champ_url).json()["data"][champion]
    return [champ_page[infos] for infos in champ_page if infos in ATTRIBUTES_CHAMP and champ_page[infos]!=[]]


In [11]:
def rag_answer(query):
    lang=language_detector(query)
    champions_list=champion_detector(query)
    infos_champions=[champ_infos(lang,champ) for champ in champions_list]

    prompt=f"""You are a very skillful player of League Of Legends.\
    You give really useful and uncommon tips to play specific champions.\
    You always use data of champions to give the tips.\
    You also analyse statistiques to give uncommon and very useful tips.\
    You always use all the knowedges to give good tips when you see a query.\
    You always answer is the language associated to {lang}\
    Here are champions: {champions_list}\
    Here are data of champions: {infos_champions}\
    Here is a query: {query}"""

    answer=llm.invoke(prompt).content
    return answer

print(rag_answer("J'adore jouer Orianna midlane mais je fais que perdre contre Akali."))

Voici quelques conseils pour jouer Orianna contre Akali :

1. Utilisez votre compétence "Ordre : Protection" pour vous protéger des attaques d'Akali. Le bouclier et les dégâts magiques infligés peuvent vous aider à contrer ses attaques.

2. Utilisez "Ordre : Dissonance" pour ralentir Akali et l'empêcher de s'approcher de vous trop facilement. Cela peut vous aider à maintenir une distance sécurisée.

3. Gardez un œil sur la position de la sphère d'Orianna et utilisez-la stratégiquement pour contrôler l'espace et empêcher Akali de s'approcher trop près.

4. Profitez de la portée de vos compétences pour harceler Akali à distance et éviter les échanges de coups directs.

En suivant ces conseils et en adaptant votre style de jeu, vous devriez être en mesure de mieux affronter Akali en tant qu'Orianna midlane. Bonne chance !


In [10]:
if __name__ == "__main__":
    print("Building retriever...")

    emb=OpenAIEmbeddings()
    llm=ChatOpenAI(
        model_name="gpt-3.5-turbo",
        temperature=0.0,
        max_tokens=500
    )

    try:
        while True:
            print('-' * 50)
            question=input("Posez une question :\n> ")
            print()
            print(rag_answer(question))
            print("\n")

    except KeyboardInterrupt:
        print("\nExiting...")

Building retriever...


C:\Users\asami\AppData\Local\Temp\ipykernel_23404\1174825200.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  emb=OpenAIEmbeddings()


--------------------------------------------------


Exiting...
